In [ ]:
r = requests.get('https://forum.awd.ru/viewtopic.php?f=521&t=397637&start=120', headers = headers)

sp = bs(r.text, 'html.parser')

sp.find_all('div', 'content')



In [ ]:
import pandas as pd
import seaborn as sns
import requests
from bs4 import BeautifulSoup as bs
import dfply
import re
import numpy as np


Функция для получения нужных страниц форума

In [ ]:
def parse_forum(url, starts, headers):
    result = []
    for start in starts:
        request = requests.get(url = url + str(start), headers = headers)
        result.append(bs(request.text, 'html.parser'))
    return result

Функция для спарсивания нужных кусков страницы (del нужно, потому что на каждой странице вверху есть опрос, который тоже спарсивается, но не нужен)

In [ ]:
def parse_soups(soups, tag, class_, delete):
    result = []
    for el in soups:
        content = el.find_all(tag, class_ = class_)
        del content[delete]
        for elem in content:
            result.append(elem.text.strip())
    return result

In [ ]:
starts = [0,20, 40, 60, 80, 100, 120, 140]
headers = {
    'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.5 Safari/605.1.15',
    'cookie': '_ga_W0JDW2V29Y=GS1.1.1661181837.2.1.1661184512.60.0.0; _ga=GA1.2.1025230771.1661155454; _gid=GA1.2.721157209.1661155454; style_cookie=null; phpbb3_alft2_k=; phpbb3_alft2_sid=fa5f9f17fa55d5a1a82d62c62159899a; phpbb3_alft2_u=1'
}

soups = parse_forum('https://forum.awd.ru/viewtopic.php?f=521&t=397637&start=', starts, headers)

In [ ]:
contents = parse_soups(soups, 'div', 'content', 0)
authors = parse_soups(soups, 'p', 'author', 0)

In [ ]:
print(len(contents))
print(len(authors))

In [ ]:
contents[120:131]

Функция для парсинга уже конкретных данных по регулярному выражению
Айди нужен, чтобы потом вручную прописать данные, которые не спарсятся, и не потерять порядок людей

In [ ]:
def get_data(regex, list, group_num = 2, hard = False, hard_regex = '(5\.)(.*)'):
    result = []
    for idd, value in enumerate(list):
        try:
            reg = re.search(regex, value).group(group_num).strip()
            
            if hard == True:
                for i in range(3):
                    try:
                        reg = re.search(hard_regex, reg).group(2).strip()
                    except:
                        break
                        
                        
            result.append(reg)
        except:
            print("id = " + str(idd) + "__Не пон " + str(value) + "\n___________")
            result.append(np.NaN)
    return result

In [ ]:
city = get_data('(1\.)(.*?)(2\.)', contents)
city

In [ ]:
visa_center = get_data('(2\.)(.*?)(3\.)', contents) 
visa_center

In [ ]:
date = get_data('\d\d\s\w{2,4}\s\d\d\d\d', authors, 0)
date

In [ ]:
visa_history = get_data('(4\.)(.*?)(5\.)', contents)
visa_history

In [ ]:
visa_limit = get_data('(5\.)(.+?)(6\.)', contents, hard = True)
visa_limit

In [ ]:
date = get_data('\d\d\s\w{2,4}\s\d\d\d\d', authors, 0)
city = get_data('(1\.)(.*?)(2\.)', contents)
visa_center = get_data('(2\.)(.*?)(3\.)', contents) 
visa_history = get_data('(4\.)(.*?)(5\.)', contents)
visa_limit = get_data('(5\.)(.+?)(6\.)', contents, hard = True)

In [ ]:
df = pd.DataFrame()

df['date'] = date
df['city'] = city
df['visa_center'] = visa_center
df['visa_history'] = visa_history
df['visa_limit'] = visa_limit
df

In [ ]:
df.isna().sum()